# Pneumonia Detection using CNN

In [1]:
from tensorflow import keras
from keras.layers import Input,Lambda , Dense , Flatten

In [2]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [3]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Model Generation

In [4]:
IMAGE_SIZE = [224,224]

train_path ="Datasets/train"
test_path = "Datasets/test"

In [5]:
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False )

In [6]:
for layer in vgg.layers : 
    layer.trainable = False

In [7]:
folders = glob('Datasets/train/*')
x = Flatten() (vgg.output)

In [8]:
prediction = Dense(len(folders),activation="softmax")(x)

model = Model(inputs=vgg.input, outputs = prediction)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
model.compile(
loss='categorical_crossentropy',
optimizer = 'adam',
metrics = ['accuracy']
)

In [10]:
from keras.preprocessing.image import ImageDataGenerator


In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip= True
                                  )

In [12]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                target_size = (224,224),
                                                batch_size = 10,
                                                class_mode ='categorical') 

Found 5216 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                           target_size= (224,224),
                                            batch_size = 10,
                                            class_mode ='categorical'
                                           )

Found 624 images belonging to 2 classes.


In [15]:
r = model.fit_generator(
training_set,
validation_data = test_set,
epochs = 1 ,
steps_per_epoch = len(training_set),
validation_steps = len(test_set)
)

C:\Users\Hepha\AppData\Local\Temp\ipykernel_19180\3779747041.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


522/522 [==============================] - 1010s 2s/step - loss: 0.2026 - accuracy: 0.9256 - val_loss: 0.2931 - val_accuracy: 0.9135


In [1]:
import tensorflow as tf
from keras.models import load_model



In [ ]:
model.save('chest_xray.h5')

## Import model 

In [2]:
model = load_model('chest_xray.h5')

In [3]:
from keras.preprocessing import image

In [4]:
from keras.applications.vgg16 import preprocess_input

In [5]:
import numpy as np

## Normal xray images : 

In [6]:
# image.load_img doesnt work so i had to use tf.keras.utils
img = tf.keras.utils.load_img('./Datasets/val/NORMAL/NORMAL2-IM-1430-0001.jpeg',target_size=(224,224))

In [7]:
x=tf.keras.utils.img_to_array(img)

In [8]:
x = np.expand_dims(x,axis=0)

In [9]:
img_data = preprocess_input(x)

In [10]:
classes= model.predict(img_data)

1/1 [==============================] - 3s 3s/step


In [11]:
result = classes[0][0]

In [12]:
if result > 0.9 : 
    print("Result is Normal")
else:
    print("Person is Affected by pneumonia")

Result is Normal


## pneumonic xray images

In [50]:
# image.load_img doesnt work so i had to use tf.keras.utils
img = tf.keras.utils.load_img('./Datasets/val/PNEUMONIA/person1946_bacteria_4875.jpeg',target_size=(224,224))

In [51]:
x=tf.keras.utils.img_to_array(img)

In [52]:
x = np.expand_dims(x,axis=0)

In [53]:
img_data = preprocess_input(x)

In [54]:
classes= model.predict(img_data)

1/1 [==============================] - 0s 153ms/step


In [55]:
result = classes[0][0]

In [56]:
if result > 0.9 : 
    print("Result is Normal")
else:
    print("Person is Affected by pneumonia")

Person is Affected by pneumonia


## Accuracy : 90%